In [5]:
import os
import sys
import pprint

base_dir = 'F:\Thesis_ssd\MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.ResultFitter import ResultFitter
import json

In [16]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.5,
    'seed' : 1,
    'subsample_size' : 0.25
}

loadData = LoadData(**load_args)

full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
dataGen = DataGenerator(loadData)
resultFitter = ResultFitter(loadData)

AttributeError: 'LoadData' object has no attribute 'getDatasets'

In [6]:
result_file_name = 'results_6_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
model = resultFitter.fit_from_csv_and_index(result_file_name, 0, 2, 10, use_tensorboard = False, 
                               use_liveplots = True, use_custom_callback = False)

NameError: name 'resultFitter' is not defined

In [34]:
result_file_name = 'results_7_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
df = GridSearchResultProcessor().get_results_df_by_name(result_file_name, 2)

In [35]:
df[df.columns[:13]]

,batch_size,epochs,learning_rate,optimizer,activation,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons
0,64,30,0.01000,sgd,tanh,0.300,17,5,0.0001,0.0010,sigmoid,same,32
1,128,33,0.10000,rmsprop,tanh,0.001,11,13,0.0010,0.2000,sigmoid,same,256
2,128,35,0.00100,rmsprop,sigmoid,0.100,19,11,0.0010,0.0010,sigmoid,same,256
3,512,35,0.01000,rmsprop,softmax,0.200,23,9,0.0010,0.0001,sigmoid,same,32
4,128,30,0.01000,sgd,tanh,0.010,15,5,0.3000,0.2000,sigmoid,same,64
5,256,30,0.00001,adam,relu,0.000,19,11,0.1000,0.1000,sigmoid,same,256
6,64,33,0.00001,rmsprop,sigmoid,0.400,13,3,0.1000,0.0010,sigmoid,same,128
7,128,33,0.00010,rmsprop,relu,0.001,23,15,0.2000,0.0010,sigmoid,same,512
8,64,50,0.01000,sgd,tanh,0.500,17,5,0.0001,0.0010,sigmoid,same,32
9,64,50,0.01000,sgd,tanh,0.500,17,5,0.0001,0.0010,sigmoid,same,32


In [36]:
df[df.columns[13:]]

,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall
0,0.580652,0.893406,0.964071,0.871845,0.753918,0.853960,0.857098,0.849133
1,3334.017578,0.651845,0.661726,0.972027,3334.112305,0.497969,0.498642,0.976846
2,3.780298,0.663522,0.663522,1.000000,3.882282,0.499375,0.499375,1.000000
3,3.926843,0.663647,0.663647,1.000000,4.042869,0.499023,0.499023,1.000000
4,697.530762,0.660738,0.662526,0.995771,697.642822,0.498906,0.499139,0.997810
5,124.681503,0.663668,0.663647,1.000000,124.788605,0.499375,0.499375,1.000000
6,2.304512,0.663513,0.663513,1.000000,2.421650,0.499381,0.499381,1.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fit_from_csv_and_index(result_file_name, 0, 2)

NameError: name 'fit_from_csv_and_index' is not defined

In [ ]:
li = fit_from_csv_and_index(result_file_name, 0, 2)

In [ ]:
mysplit(result_fi)

In [60]:
class A(B):
    def __init__(self, hey):
        self.hey = hey

class B:
    def __init__(self):
        pass
    def check(self):
        print(hasattr(self, 'hey'))

In [61]:
A = A("bye")

In [62]:
A.check()

AttributeError: 'A' object has no attribute 'check'

In [58]:
hasattr(A, 'hey')

True